In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from psql_helpers import read_table_async

In [3]:
from psql_models import *

In [4]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


In [5]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
#await reset_content_indexing()

In [9]:
df = await read_table_async(Content)

In [10]:
df

,id,type,text,timestamp,parent_id,is_indexed,last_update
0,dfa86d3e-4946-414d-bd66-bd0ea472dc5c,meeting,None,2024-06-05 12:56:23.602416,None,False,2024-06-05 12:46:02.777662
1,f858cb3e-ac65-467c-8f5f-d0704abff5d3,meeting,None,2024-06-18 11:00:08.984681,None,False,2024-06-04 10:36:58.015811
2,f57553c1-1b73-47e6-97e9-ae3d5295d6d4,meeting,None,2024-06-10 15:09:50.271597,None,False,2024-06-10 14:16:30.597309
3,aa6f0a6a-60a4-4a14-a005-78a377ad6b15,meeting,None,2024-06-11 13:28:41.232765,None,False,2024-06-11 12:47:32.693585
4,167773e6-12f1-41b1-af82-8fa586fc83f7,meeting,None,2024-06-11 15:00:47.108002,None,False,2024-06-11 15:23:09.096777
...,...,...,...,...,...,...,...
4942,3ccc968c-4442-4db9-bd53-906c699b480c,meeting,None,2024-07-16 12:30:12.646382,None,True,2024-07-16 12:30:17.431634
4943,6ce92345-80f9-4273-8f5b-3ae947a15d70,meeting,None,2024-08-08 09:25:20.482626,None,True,2024-08-08 10:27:10.631886
4944,bbceeaf7-4c9b-4c68-b6b4-9d1a0086fb88,meeting,None,2025-01-29 21:52:21.000000,None,False,2025-01-29 21:55:46.000000
4945,8ad79505-744f-4e6b-be15-d80d3be7e1ed,title,22:04,NaT,5ff0dfc4-6a04-4fd6-8ead-9cd93eb30b87,False,NaT


In [8]:
df[df['is_indexed']]

KeyError: 'is_indexed'

In [19]:
df = await read_table_async(Content)

In [20]:
df

""


In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3683
Name: count, dtype: int64

In [9]:
df['type'].value_counts()


type
meeting    3683
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined